<a href="https://colab.research.google.com/github/MasslessAI/narratelab/blob/master/report_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('gdrive', force_remount=True)

root_dir = '/content/gdrive/'

gdrive_path = root_dir + 'MyDrive/narratelab/reports'

!pip install spacy==3.1.0 colour sentence-transformers redditcleaner psaw pandas loguru distinctipy tqdm praw asyncpraw rake-spacy hdbscan umap-learn==0.5.1
!python -m spacy download en_core_web_trf

Mounted at gdrive
     |████████████████████████████████| 6.4 MB 7.5 MB/s 
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 57 kB 6.4 MB/s 
     |████████████████████████████████| 167 kB 72.9 MB/s 
     |████████████████████████████████| 172 kB 70.6 MB/s 
     |████████████████████████████████| 6.4 MB 25.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 80 kB 11.6 MB/s 
     |████████████████████████████████| 10.1 MB 43.1 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 456 kB 55.0 MB/s 
     |████████████████████████████████| 623 kB 49.9 MB/s 
     |████████████████████████████████| 1.1 MB 69.5 MB/s 
     |████████████████████████████████| 2.8 MB 62.5 MB/s 
     |████████████████████████████████| 1.2 MB 46.4 MB/s 
     |████████████████████████████████| 50 kB 7.7 M

In [2]:
import os
path = "/content" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

!pip uninstall bertopic --yes
!rm -fr BERTopic
!git clone https://github.com/MasslessAI/BERTopic.git
%cd BERTopic

# 2. install the project/module
!python setup.py install

# 3. Add the project directory to the path
import os, sys
sys.path.append(os.getcwd())

%cd ..

Cloning into 'BERTopic'...
remote: Enumerating objects: 1125, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1125 (delta 111), reused 77 (delta 71), pack-reused 951
Receiving objects: 100% (1125/1125), 7.02 MiB | 19.16 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/BERTopic
running install
running bdist_egg
running egg_info
creating bertopic.egg-info
writing bertopic.egg-info/PKG-INFO
writing dependency_links to bertopic.egg-info/dependency_links.txt
writing requirements to bertopic.egg-info/requires.txt
writing top-level names to bertopic.egg-info/top_level.txt
writing manifest file 'bertopic.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'bertopic.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/tests
copying tests/test_topic_representation.py -> build/li

In [3]:
import datetime as dt
from datetime import date, datetime
from loguru import logger
import pandas as pd
from psaw import PushshiftAPI
import time
import os
import redditcleaner
import re
import base64
import IPython
from distinctipy import distinctipy
import random
from colour import Color
import praw
import math
from collections import defaultdict

r = praw.Reddit(
  client_id="_tkKH5nva25sWQ",
  client_secret="cGMI8KyRQlopBTx7vGAEbgCDF6RzwQ",
  user_agent="analysis",
  check_for_async=False
)


api = PushshiftAPI(r)

In [13]:
# setup parameters
subreddit_count_start_epoch = int(dt.datetime(2020, 1, 1).timestamp())
subreddit_count_end_epoch = int(time.time())

# (optional) Search for the most relevant subreddits
KEYWORDS = ["internal communication", "product management", "leadership", "business", "enterprise", "technology", "ux"]
keywords_query = '|'.join(map(lambda x: '"{}"'.format(x), KEYWORDS))

subreddit_count_total = 0
subreddit_count = defaultdict(lambda: 0)
while True:
    gen = list(
        api.search_submissions(
            after=subreddit_count_start_epoch, 
            before=subreddit_count_end_epoch,
            q=keywords_query,
            #subreddit='cryptomoonshots',
            score=">10",
            filter=['subreddit', 'created_utc', 'title', 'selftext', 'score', 'permalink'],
            sort='asc', 
            sort_type='created_utc', 
            limit=500))
    
    for item in gen:
      subreddit_count[item.subreddit] += 1


    if len(gen) == 0:
        break

    if len(gen) > 0:
      subreddit_count_start_epoch = int(gen[-1].created_utc)
      subreddit_count_total += len(gen)

      logger.info('Searched {} Total {} Last created_utc {}'.format(
        len(gen), subreddit_count_total, date.fromtimestamp(subreddit_count_start_epoch)))

      time.sleep(3)
    else:
      break

subreddit_count_df = pd.DataFrame(subreddit_count.items(), columns=['subreddit', 'count']).sort_values(by=['count'], ascending=False)
print(subreddit_count_df.head(50))

2021-09-06 21:15:12.511 | INFO     | __main__:<module>:36 - Searched 484 Total 484 Last created_utc 2020-01-03
2021-09-06 21:15:24.521 | INFO     | __main__:<module>:36 - Searched 477 Total 961 Last created_utc 2020-01-04
2021-09-06 21:15:38.521 | INFO     | __main__:<module>:36 - Searched 470 Total 1431 Last created_utc 2020-03-29
2021-09-06 21:15:51.360 | INFO     | __main__:<module>:36 - Searched 489 Total 1920 Last created_utc 2020-04-11
2021-09-06 21:16:04.947 | INFO     | __main__:<module>:36 - Searched 486 Total 2406 Last created_utc 2020-04-12
2021-09-06 21:16:19.976 | INFO     | __main__:<module>:36 - Searched 489 Total 2895 Last created_utc 2020-04-13
2021-09-06 21:16:33.078 | INFO     | __main__:<module>:36 - Searched 492 Total 3387 Last created_utc 2020-04-14
2021-09-06 21:16:49.279 | INFO     | __main__:<module>:36 - Searched 490 Total 3877 Last created_utc 2020-04-15
2021-09-06 21:17:04.598 | INFO     | __main__:<module>:36 - Searched 481 Total 4358 Last created_utc 2020-

                subreddit  count
447        wallstreetbets    949
106                   HFY    548
27                  memes    507
10          AmItheAsshole    479
263              politics    474
34             conspiracy    393
82       unpopularopinion    338
1934          pennystocks    303
3450      CryptoMoonShots    275
6149                  GME    260
8699           Superstonk    244
7                 nosleep    242
1362             dogecoin    234
6421             amcstock    229
589             AzureLane    216
84                 stocks    211
22          smallbusiness    184
95         CryptoCurrency    179
79              JUSTNOMIL    158
425             dankmemes    153
422             investing    151
2864                SPACs    148
6314     Wallstreetsilver    148
323         SquaredCircle    143
426              exmormon    136
533     ApplyingToCollege    123
207                 india    121
628             childfree    119
55           Entrepreneur    115
266   rela

In [4]:
# setup parameters
start_epoch = int(dt.datetime(2015, 1, 1).timestamp())
end_epoch = int(time.time())

# SUBMISSIONS_FILE = gdrive_path + '/reddit_submission_entrepreneur_2015_01_01_2021_08_17.tsv'
SUBMISSIONS_FILE = None
SUBREDDIT = 'plasticsurgery'
DATA_FILE_NAME = gdrive_path + '/reddit_submission_{}_{}_{}.tsv'.format(
    SUBREDDIT, datetime.fromtimestamp(start_epoch).strftime("%Y_%m_%d"),
    datetime.fromtimestamp(end_epoch).strftime("%Y_%m_%d"))


QUESTION_WORDS = [
  "what",
  "when",
  "where",
  "who",
  "whom",
  "which",
  "whose",
  "why",
  "how",
  "wonder",
  "want",
  "is anyone",
  "does anyone",
  "any tips",
  "advice",
  "suggestion",
  "suggestions",
  "suggest",
  "ideas on",
  "need help",
  "needs help",
  "need your help",
  "serious help",
  "please help",
  "challenge",
  "challenges",
  "can't stand",
  "struggle",
  "struggling",
  "can't figure out",
  "help me",
  "hardest part",
  "would appreciate",
  "would really appreciate",
  "any guidance",
  "no idea",
  "confused with",
  "new to",
  "is there any way"
]

total = 0
title_query = '|'.join(map(lambda x: '"{}"'.format(x), QUESTION_WORDS))

if SUBMISSIONS_FILE is None:
  # skip crawling if file already specified
  SUBMISSIONS_FILE = DATA_FILE_NAME
  while True:
      gen = list(
          api.search_submissions(
              after=start_epoch, 
              before=end_epoch,
              title=title_query,
              is_self=True,
              is_original_content=True,
              subreddit=SUBREDDIT,
              # note the num_comments is only updated for the first 24hr
              # may need to use praw to get latest meta data
              num_comments=">1",
              filter=['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received',
                      'upvote_ratio'],
              sort='asc', 
              sort_type='created_utc', 
              limit=500))

      if len(gen) == 0:
          break

      def submission_filter(submission):
          if 'title' not in submission:
              return False
          if 'selftext' not in submission:
              return False
          if len(submission['selftext']) == 0:
              # if submission is deleted, the psaw returns empty str
              return False
          if 'author' not in submission:
              return False
          if submission['author'] == "[deleted]":
              return False
          if any(submission['selftext'] == x for x in ["[removed]", "[deleted]"]):
              return False
          return True

      def prepare_data(data):
          # some of the fields may be missing
          # must manually set an init value to avoid
          # generating invalid csv
          _data = {
              'title': '',
              'selftext': '',
              'author': '',
              'permalink': '',
              'num_comments': 0,
              'score': 0,
              'total_awards_received': 0,
              'upvote_ratio': 1.0,
              'created_utc': None
          }

          for key in _data:
              if key in data and data[key] is not None:
                  _data[key] = data[key]

          return _data

      
      items = []
      keys = ['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received', 'upvote_ratio', 'created_utc']
      for _item in gen:
        items.append({
          'title': _item.title,
          'selftext': _item.selftext,
          'author': _item.author,
          'permalink': _item.permalink,
          'num_comments': _item.num_comments,
          'score': _item.score,
          #'total_awards_received': _item.total_awards_received,
          'upvote_ratio': _item.upvote_ratio,
          'created_utc': _item.created_utc
        })

      items = map(prepare_data, items)
      # items = map(prepare_data, [item.d_ for item in gen])

      items = list(filter(submission_filter, items))
      df = pd.DataFrame(items)

      # clean data
      def clean(text):
          text = text.lower()
          # remove reddit styles
          text = redditcleaner.clean(
              text, quote=False, bullet_point=False, link=False, strikethrough=False, spoiler=False, code=False,
              superscript=False, table=False)

          # refer to https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76
          # Remove unicode characters
          text = text.encode('ascii', 'ignore').decode()

          # Remove Hashtags
          text = re.sub("#\S+", " ", text)

          # Remove markdown links
          text = re.sub(r"\[(.+)\]\(.+\)", r"\1", text)

          # Remove other urls
          text = re.sub(r"http\S+", " ", text)

          # remove text inside brackets
          text = re.sub("\(.*?\)"," ", text)
          text = re.sub("\[.*?\]"," ", text)

          # remove quotes
          # remove brackets
          # remove semicolon
          text = re.sub(r'[\t()[\]\"*:\\]',' ', text)

          # remove non-ascii chars
          text = re.sub(r"[^\x00-\x7F]+",' ', text)

          # remove x200b zero-width space
          text = re.sub(r"x200b",'', text)

          # Replace the over spaces# if submission is deleted, the psaw returns NaN, must check if 
              # it's a valid string
          text = re.sub('\s{2,}', " ", text)

          return text

      if len(df) > 0:
        df['title'] = df['title'].map(clean)
        df['selftext'] = df['selftext'].map(clean)

        if not os.path.isfile(DATA_FILE_NAME):
            df.to_csv(DATA_FILE_NAME, sep='\t', header='column_names', index=False, quoting=3)
        else:  # else it exists so append without writing the header
            df.to_csv(DATA_FILE_NAME, sep='\t', mode='a', header=False, index=False, quoting=3)

        start_epoch = int(items[-1]['created_utc'])
        total += len(items)

        logger.info('Added {} Total {} Last created_utc {}'.format(
          len(items), total, date.fromtimestamp(start_epoch)))

        time.sleep(3)
      else:
        break

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
2021-09-06 19:35:03.172 | INFO     | __main__:<module>:197 - Added 233 Total 233 Last created_utc 2017-10-05
2021-09-06 19:35:15.122 | INFO     | __main__:<module>:197 - Added 195 Total 428 Last created_utc 2018-08-31
2021-09-06 19:35:26.073 | INFO     | __main__:<module>:197 - Added 207 Total 635 Last created_utc 2019-03-12
2021-09-06 19:35:37.091 | INFO     | __main__:<module>:197 - Added 174 Total 809 Last created_utc 2019-08-21
2021-09-06 19:35:47.887 | INFO     | __main__:<module>:197 - Added 210 Total 1019 Last created_utc 2020-01-10
2021-09-06 19:36:01.829 | INFO     | __main__:<module>:197 - Added 209 Total 1228 Last created_utc 2020-06-01
2021-09-06 19:36:15.098 | INFO     | __main__:<module>:197 - Added 246 Total 1474 Last created_utc 2020-09-02
2021-09-06 19:36:26.122 | INFO     | __main__:<mod

In [5]:
df = pd.read_csv(SUBMISSIONS_FILE, sep='\t', quoting=3)

# data cleanup
print('before clean-up # rows: {}'.format(len(df)))
cleaned_rows = []

AD_INDICATIVE_PHRASES = [
  "your business", 
  "your businesses", 
  "help you", 
  "case study"
  "how i",
  "step-by-step",
  "here is how",
  "here's how",
  "part 1",
  "part 2",
  "part 3",
  "ultimate guide",
  "cheatsheet",
  "infographic",
  "ama"
]

# clear out all rows with missing values
df = df.dropna()

for index, row in df.iterrows():
    row_title = row['title'].lower()
    if not any(x in [':', ';'] for x in row_title) and not any(phrase in row_title for phrase in AD_INDICATIVE_PHRASES) and '?' in row_title:
        '''
        1. must contain '?'
        2. can only contain alphanumeric, punctuations and space
        3. should not contain '-', ':', ';' which indicates ads
        '''
        cleaned_rows.append(index)                                                       

df = df[df.index.isin(cleaned_rows)].reset_index()
print('after clean-up # rows: {}'.format(len(df)))

before clean-up # rows: 2712
after clean-up # rows: 1839


In [6]:
import pprint
import spacy
from tqdm import tqdm 
from rake_spacy import Rake

spacy.require_gpu()
nlp = spacy.load("en_core_web_trf")
rake = Rake(nlp=nlp)

title_cat = []
keywords = []
lemmatized_combined_text = []

# handle cases where selftext is nan
df['selftext'] = df['selftext'].map(lambda text: text if type(text) == 'string' else "")

df['combined_text'] = df['title'] +' '+ df['selftext']
for index, row in tqdm(df.iterrows(), position=0, leave=True, total=len(df)):
    title_cat.append('NO_WH_WORD')  
    for wh_word in QUESTION_WORDS:
        if wh_word in row['title'].lower():
            title_cat[-1] = wh_word
            break

df['title_cat'] = title_cat

cat_stats = {
    'cat': QUESTION_WORDS,
    'num_docs': [],
    'total_score': [],
    'total_comments': []
}

# print out number of docs per category
for wh_word in QUESTION_WORDS:
    _df = df[df['title_cat'] == wh_word]
    cat_stats['num_docs'].append(len(_df))
    cat_stats['total_score'].append(_df['score'].sum())
    cat_stats['total_comments'].append(_df['num_comments'].sum())
  
cat_stats_df = pd.DataFrame(data=cat_stats)

print('\n', cat_stats_df.to_markdown())
  

100%|██████████| 1839/1839 [00:00<00:00, 10523.20it/s]



 |    | cat                     |   num_docs |   total_score |   total_comments |
|---:|:------------------------|-----------:|--------------:|-----------------:|
|  0 | what                    |        553 |          2474 |             4397 |
|  1 | when                    |        101 |           446 |              689 |
|  2 | where                   |         53 |           208 |              314 |
|  3 | who                     |        134 |          1115 |             1259 |
|  4 | whom                    |          0 |             0 |                0 |
|  5 | which                   |         46 |           160 |              318 |
|  6 | whose                   |          0 |             0 |                0 |
|  7 | why                     |         77 |           518 |              734 |
|  8 | how                     |        604 |          2787 |             4937 |
|  9 | wonder                  |          0 |             0 |                0 |
| 10 | want               

In [7]:
# !pip uninstall umap-learn -y
# !pip install 'umap-learn==0.5.1'

In [8]:
import sys
sys.path.insert(0,'/content/BERTopic')

In [9]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic.backend import BaseEmbedder


sentence_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# we must use BaseEmbedder as base class, otherwise BERTopic will use default 
# SentenceTransformer("paraphrase-MiniLM-L6-v2")
class CustomSentenceModel(BaseEmbedder):
  def __init__(self, embedding_model):
    super().__init__()
    self.embedding_model = embedding_model
  
  def embed(self, documents, verbose):
    embeddings = self.encode(documents, show_progress_bar=verbose)
    return embeddings

  def encode(self,documents,show_progress_bar = False):
        # need to split phrases into individual words in order to use general
        # sentence embedding models

        # some doc can be NaN, ignore these
        documents = list(map(lambda doc: re.sub('_', ' ', doc), documents))
        
        embeddings = self.embedding_model.encode(documents, show_progress_bar=show_progress_bar)
        return embeddings

def preprocess_text(documents_by_cluster):
  processed_documents_by_cluster = []
  for document in documents_by_cluster:
    docs = list(nlp.pipe(document.split('\t')))
    processed_doc = []
    for doc in docs:
       # merge entities and noun phrases (without DET)
      def valid_start_pos(doc, span):
        start = span.start
        while start < span.end:
          if doc[start].pos_ not in ['DET', 'PRON']:
            break
          start += 1
        return start

      # remove prefix tokens that are either DET or PRON
      # remove spans with less than 2 tokens
      # remove spans with 0 tokens after removing prefix
      noun_phrase_spans = list(filter(lambda x: x.start < x.end, [doc[valid_start_pos(doc, np_span):np_span.end] for np_span in doc.noun_chunks if np_span.end-np_span.start > 1]))
      entities_spans = list(filter(lambda x: x.start < x.end, [doc[valid_start_pos(doc, ent_span):ent_span.end] for ent_span in doc.ents if ent_span.end-ent_span.start > 1]))

      spans = spacy.util.filter_spans(noun_phrase_spans+entities_spans)
      with doc.retokenize() as retokenizer:
        for span in spans:
          retokenizer.merge(span)
      
      processed_text = ' '.join(['_'.join(token.lemma_.split(' ')) for token in doc if not token.is_stop])
      processed_doc.append(processed_text)
    processed_documents_by_cluster.append('\t'.join(processed_doc))
  return processed_documents_by_cluster

df_selection = df.copy().reset_index()
_docs = df_selection['combined_text']

vectorizer = CountVectorizer(ngram_range=(1,1), max_df=0.95)

topic_model = BERTopic(preprocess_text=preprocess_text, 
                       vectorizer_model=vectorizer,
                       embedding_model=CustomSentenceModel(sentence_model),
                       nr_topics="auto", calculate_probabilities=True,
                       min_topic_size=5)

topics, probabilities = topic_model.fit_transform(_docs)
topic_info_df = topic_model.get_topic_info()
topic_tokens = [[token[0] for token in topic_model.get_topic(topic=row['Topic'])] for idx, row in topic_info_df.iterrows()]
topic_info_df['Tokens']=topic_tokens
print(topic_info_df[['Topic', 'Count', 'Tokens']].to_markdown())

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

2021-09-06 19:37:43,953 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning:

The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.

2021-09-06 19:37:57,809 - BERTopic - Reduced dimensionality with UMAP
2021-09-06 19:37:58,324 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-09-06 19:38:21,328 - BERTopic - Reduced number of topics from 69 to 31


|    |   Topic |   Count | Tokens                                                                                                                                                                                                                  |
|---:|--------:|--------:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 |      -1 |     443 | ['face', 'filler', 'work', 'implant', 'plastic_surgeon', 'plastic_surgery', 'look', 'doctor', 'go', 'feel']                                                                                                             |
|  1 |       0 |     176 | ['lip_filler', 'lip', 'lipo', 'liposuction', 'lip_injection', 'under_eye_filler', 'chin_lipo', 'lip_lift', 'eye_filler', 'dermal_filler']                                                                               |
|  2 |       1 |    

In [10]:
# dump topic info
REPORT_TOPIC_FILE = SUBMISSIONS_FILE.replace("reddit_submission", "report_topic")
REPORT_TOPIC_FILE = REPORT_TOPIC_FILE.replace(".tsv", ".json")
topic_info_df.to_json(REPORT_TOPIC_FILE)

# add topic data to doc then dump
doc_topic = []
doc_topic_prob = []
for i in range(len(topics)):
  doc_topic.append(topics[i])
  doc_topic_prob.append(probabilities[i][topics[i]])

df_selection['topic'] = doc_topic
df_selection['topic_prob'] = doc_topic_prob

REPORT_SUBMISSION_FILE = SUBMISSIONS_FILE.replace("reddit_submission", "report_submission")
REPORT_SUBMISSION_FILE = REPORT_SUBMISSION_FILE.replace(".tsv", ".json")
df_selection.to_json(REPORT_SUBMISSION_FILE)


In [11]:
# dump topic over time
df_topics_over_time = topic_model.topics_over_time(df_selection['title'], topics, df_selection['created_utc'], nr_bins=50)

REPORT_TOPIC_OVER_TIME_FILE = SUBMISSIONS_FILE.replace("reddit_submission", "report_topic_over_time")
REPORT_TOPIC_OVER_TIME_FILE = REPORT_TOPIC_OVER_TIME_FILE.replace(".tsv", ".json")
# print(df_topics_over_time.to_markdown())
df_topics_over_time.to_json(REPORT_TOPIC_OVER_TIME_FILE)